<a href="https://colab.research.google.com/github/choarauc/form/blob/main/methodologie_deep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Charger le jeu de données 'face_images.csv' dans le dataframe df.
Faire un bref audit des données.

In [ ]:
import pandas as pd
df = pd.read_csv('face_images.csv')
df.head()

Pour rappel, les images sont toutes stockées dans un dossier "images". Et, pour chaque image, df renseigne son nom et les coordonnées normalisées de l'objet. xmin, ymin, xmax et ymax représentent les coordonnées normalisées de la "bounding box" de l'objet, c'est-à-dire la boite qui englobe l'objet.



xminxmin  : position horizontale de l'extrémité gauche de la boîte.
yminymin  : position verticale de l'extrémité gauche de la boîte.
xmaxxmax  : position horizontale de l'extrémité droite de la boîte.
ymaxymax  : position verticale de l'extrémité droite de la boîte.

Définir une fonction load_image avec comme argument filepath et resize qui charge, redimensionne et qui retourne l'image. Utiliser uniquement des fonctions de tensorflow.

Charger et afficher la première image de df.
Afficher ses dimensions.

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
folder_images = 'images/'

def load_image(filepath, resize=None):
    im = tf.io.read_file(folder_images + filepath)
    im = tf.image.decode_png(im, channels=3)
    if resize:
        return tf.image.resize(im, resize)
    else :
        return im

        
im = load_image(df.filename[0])
plt.imshow(im);


Pour localiser une unique boîte encadrant le visage dans l'image, nous pouvons nous ramener à un problème de régression sur les 4 variables cibles suivantes.



xmoyxmoy  : position horizontale (normalisée) du milieu de la boîte.
ymoyymoy  : position verticale (normalisée) du milieu de la boîte.
ww  : largeur (normalisée) de la boîte.
hh  : hauteur (normalisée) de la boîte.

Créer les colonnes suivantes à df : xmoy, ymoy, w et h.
Afficher les premières lignes de df.

In [ ]:
df['xmoy'] = (df.xmax + df.xmin)/2
df['ymoy'] = (df.ymax + df.ymin)/2
df['w'] = (df.xmax - df.xmin)
df['h'] = (df.ymax - df.ymin)
df.head()

Exécuter la cellule suivante pour afficher la bounding box de la première image.

In [ ]:
import numpy as np

def show_bounding_box(im, bbox, normalised=True, color='r'):
    # Signification de bbox
    x, y, w, h = bbox
    # Convertir les cordonées (x,y,w,h) en (x1,x2,y1,y2)
    x1=x-w/2
    x2=x+w/2
    y1=y-h/2
    y2=y+h/2
    
    # Redimentionner en cas de normalisation
    if normalised:
        x1=x1*im.shape[1]
        x2=x2*im.shape[1]
        y1=y1*im.shape[0]
        y2=y2*im.shape[0]
        
    # Afficher l'image
    plt.imshow(im)
    
    # Afficher la bounding box
    plt.plot([x1,x2,x2,x1,x1],[y1,y1,y2,y2,y1],"r")

idx = 1
# Array de l'image
im = load_image(df.filename[idx])
# Coordonnées de la bounding box
bbox = df[['xmoy', 'ymoy', 'w', 'h']].values[idx]
# Afficher l'image ainsi que la bounding box
show_bounding_box(im, bbox)

À l'aide des fonctions définies précédemment, charger une image et afficher sa bounding box.

In [ ]:
# Array de l'image
im = load_image(df.filename[8])
# Coordonnées de la bounding box
bbox = df[['xmoy', 'ymoy', 'w', 'h']].values[8]
# Afficher l'image ainsi que la bounding box
show_bounding_box(im, bbox, normalised=True)


Exécuter la cellule suivante pour afficher aléatoirement des images avec leur bounding box.

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12,5))
for j, i in enumerate(np.random.randint(0, len(df), size=[8])):
    plt.subplot(2,4,j+1)
    plt.axis('off')
    im = load_image(df.filename[i])
    bbox = df[['xmoy', 'ymoy', 'w', 'h']].values[i]
    show_bounding_box(im, bbox, normalised=True)

Distribution des variables cibles.

Une étape importante dans un problème de régression est d'étudier la distribution de nos variables cibles pour vérifier si certaines valeurs ne sont pas sous-représentées.

Afficher la distribution de xmoy et ymoy.
Où se trouve généralement la bounding box dans l'image ?

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.displot(df.xmoy)
plt.show()
sns.displot(df.ymoy)
plt.show()
## La bounding box se trouve généralement au centre de l'image.

Afficher la distribution de w et h.
Est-ce que les petits objets sont bien représentés ?

In [ ]:
sns.displot(df.w)
plt.show()
sns.displot(df.h)
plt.show()
## Les objets ont généralement la même taille et les petits objets ne sont pas beaucoup réprésentés.


Charger le jeu de données
Le jeu de données originel est composé de 150 000 images. Pour charger l'ensemble des images en mémoire, il est alors nécessaire d'avoir au moins 40 Go de RAM et prend 2 heures de chargement. Deux solutions sont disponibles :

Comme le jeu de données est trop lourd, choisir un sous-échantillon (ex : 30 000 images) et le charger en mémoire.
Charger les images pendant l'entraînement à l'aide d'un générateur.
Il est possible de paralléliser le chargement/preprocessing des données et l'entraînement du modèle. Dans cette configuration, le modèle chargera le prochain batch en même temps que l'entraînement du modèle sur le batch actuel.

Dans le cas des images, il est possible d'appliquer ces étapes sans ralentir l'entraînement du modèle. C'est pourquoi, il est toujours préférable de choisir la seconde option qui ne limitera pas la taille du jeu de données.

Comme le jeu de test est généralement plus petit et testé à chaque epoch, il est par contre préférable de le charger en mémoire.

Séparer le jeu de données df.filepath et les variables cibles en un ensemble d'entraînement X_train_path, y_train, et en un ensemble de test X_test_path, y_test. Nous choisirons un rapport de 80% pour les données d'entraînements et une graine aléatoire 1234.
Charger les images de X_test_path redimentionnées à [256,256,3] en mémoire dans la variable X_test.

In [ ]:
from sklearn.model_selection import train_test_split
from tqdm import tqdm

X_train_path, X_test_path, y_train, y_test = train_test_split(df.filename, df[['xmoy', 'ymoy', 'w', 'h']], train_size=0.8, random_state=1234)

X_test = []
for p in tqdm(X_test_path):
    im = load_image(p, (256,256)).numpy().astype(np.uint8)
    X_test.append(im)
    
X_test = np.array(X_test)


Maintenant les données de test chargées, il est nécessaire de définir un générateur permettant de charger les images à chaque itération du modèle. Pour optimiser le temps de chargement des images en mémoire, il est possible de paralléliser le chargement de chaque image en mémoire en utilisant une structure de multi-thread.

Les objets de type dataset sur tensorflow sont capables de le faire à l'aide de l'argument num_parallel_calls de la méthode map.

Pour rappel, le constructeur from_tensor_slices du package tensorflow.data.Dataset permet de convertir une liste d'array en un dataset.

dataset = tf.data.Dataset.from_tensor_slices((X_path, y))
La méthode map du dataset permet d'appliquer une fonction à chaque observation de celui-ci. Exemple :

dataset = dataset.map(lambda x, y : [load_image(x), y])
La méthode batch du dataset permet de regrouper les observations sous forme de batch.

dataset = dataset.batch(batch_size)

Définir un dataset dataset_train de (X_train_path, y_train) à l'aide de la fonction from_tensor_slices.

À l'aide de la méthode map, appliquer la fonction load_image à chaque valeur de X_train_path. Pour que le chargement s'éffectue en multi-thread, préciser l'argument num_parallel_calls égale à -1.

Regrouper les observations sous forme de batch de taille 32.

In [ ]:
import tensorflow as tf

@tf.function
def load_image(filepath, resize=(256,256)):
    im = tf.io.read_file(folder_images+filepath)
    im = tf.image.decode_png(im, channels=3)
    return tf.image.resize(im, resize)

dataset_train = tf.data.Dataset.from_tensor_slices((X_train_path, y_train))

dataset_train = dataset_train.map(lambda x, y : [load_image(x), y], num_parallel_calls=-1).batch(32)

Exécuter la cellule suivante pour comparer le temps de chargement entre une méthode single-threading et multi-threading.

In [ ]:
import time
import cv2
t0 = time.time()
for p in X_test_path.values[:32]:
    load_image(p)

print('Time to load 32 image in a single thread method :', time.time()-t0, 's')

iterator= iter(dataset_train.take(1))
t0 = time.time()
next(iterator)
print('Time for a multi-threading method :', time.time()-t0, 's')

#Le gain observé entre une méthode single threading et multi-treading est de l'ordre de x10. 
#Il est donc beaucoup plus judicieux de charger plusieurs images simultanément qu'une seule à la fois.

Définition du modèle de détection d'objet 

Pour rappel, le modèle doit prédire à l'aide de l'image les coordonnées de la bounding box : xmoy, ymoy, w et h.

[xmoy,ymoy,w,h]=model(image)
 
Les modèles de classification d'image ou de détection d'objet utilisent généralement une approche par transfer Learning.

Quelques rappels sur le transfer leaning
L'apprentissage par transfert est le phénomène par lequel un apprentissage nouveau est facilité grâce aux apprentissages antérieurs partageant des similitudes. Par exemple, les connaissances acquises lors de l’apprentissage de la reconnaissance des voitures peuvent s’appliquer lorsqu’on essaie de reconnaître des camions.

Les modèles existants (VGG, ResNet, ...) sont composés de deux grandes parties. La première appelée backbone est un ensemble de convolution permettant l'extraction des features de l'image. La seconde est une succession de dense layer qui a pour but de classifier.

Les données du nouveau problème doivent être assez semblables avec le jeu de données utilisé pour le pré-entrainement. Dans ce cas, la méthode de transfer learning consiste à utiliser le backbone d'un modèle pré-entraîné comme extraction de features. Ensuite, des couches Dense sont ajoutées pour traiter le problème de classification ou de régression.

Lors du début de l'apprentissage, il est nécessaire de "freezer" (bloquer) les poids de la partie pré-entrainée (backbone) puisqu'ils sont proches des poids optimaux. Puis, au courant de l'entraînement, on peut "unfreeze" les couches pour affiner les poids du modèle : cette opération est appelée le fine-tuning.



Dans cet exercice, le modèle pré-entraîné sera le EfficientNet puisqu'il a montré de très bon résultat et de très bonnes propriétés pour le transfer learning.

Voici un exemple pour charger et freeze les poids d'une modèle pré-entraîner :

vgg16 = VGG16(include_top=False, input_shape=(256,256,3))
for layer in vgg16.layers:
    layer.trainable = False
model = Sequential()
model.add(vgg16)

Charger le modèle EfficientNetB0 de tensorflow.keras.applications.
Freezer les poids du modèle.
Afficher le résumé du modèle.

In [ ]:
from tensorflow.keras.applications import EfficientNetB0

# Load the model efficientNet
efficientNet = EfficientNetB0(include_top=False, input_shape=(256,256,3))

# Freeze the blackbone
for layer in efficientNet.layers:
    layer.trainable = False


Partie régression
Ajouter le modèle à un objet Sequential qui portera le nom de model.
Ajouter à ce modèle une couche GlobalAveragePooling2D.
Puis, ajouter quelques couches Dense et Dropout.
Finir par une couche Dense avec 4 neurones et une fonction activation 'linear'.

In [ ]:
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Conv2D, MaxPooling2D, BatchNormalization, LeakyReLU, Flatten
from tensorflow.keras.models import Model, Sequential, load_model

model = Sequential()
model.add(efficientNet)
model.add(GlobalAveragePooling2D())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(4, activation='linear'))
model.summary()

Définition de la fonction de perte
Quantifier l'erreur d'un modèle n'est pas toujours chose aisée. En effet, sur notre problématique, faut-il quantifier différemment l'erreur sur les coordonnées que sur la largeur/hauteur de l'objet ? Faut-il dans un premier temps enlever l'erreur sur la largeur/hauteur, pour laisser le modèle se concentrer sur les coordonnées ? Comme nous sommes face à une problématique de régression simple, l'erreur quadratique moyenne peut-elle simplement convenir ?

Il n'y a pas de réponse simple à ce type de question, c'est très souvent l'expérience et les itérations qui y répondront.

Définir une fonction de perte loss_function de votre choix avec comme argument y_true et y_pred. Attention à bien n'utiliser que des fonctions venant du package tensorflow.
Compiler le modèle avec votre fonction de perte loss_function et avec un optimizer 'adam'.

In [ ]:
from tensorflow.keras.optimizers import SGD, Adam, RMSprop

lambda_regression=10

def loss_function(y_true, y_pred):
    return lambda_regression*tf.reduce_mean(tf.square(y_true-y_pred), axis=-1)

# def loss_function(y_true, y_pred):
#     return lambda_coord*tf.reduce_mean(tf.square(y_true[...,:2]-y_pred[...,:2]), axis=-1) + lambda_largeur*tf.reduce_mean(tf.square(y_true[...,2:4]-y_pred[...,2:4]), axis=-1)


model.compile(loss=loss_function, optimizer=Adam(1e-3))

Entraînement du modèle
Entraîner le modèle à l'aide la méthode fit_generator sur 20 epochs.

In [ ]:
model.fit_generator(dataset_train, epochs=20)

Tester le modèle sur images de X_test en affichant la bounding box associée.

In [ ]:
import time
def show_img(img, model):
    plt.imshow(img)
    t0=time.time()
    x, y, w, h = model.predict(np.array([img], dtype=np.float32))[0]
    print(x, y, w, h)
    print("Execution time :",time.time()-t0,"secondes")
    show_bounding_box(img/255, [x,y,w,h])
    plt.show()
    
## Exemple :
show_img(X_test[3], model)

Exécuter la cellule suivante pour activer la webcam sur le notebook jupyter. Le code javascript va enregistrer l'image venant de votre webcam à chaque fois que le bouton Snap Photo sera pressé. L'image sera stockée dans la variable imageWebCam.

In [ ]:
%matplotlib inline
from IPython.display import HTML
from PIL import Image
import base64, io
import numpy as np

main_text = """
<style type="text/css">
    canvas {
        display: none;
    }
    </style>

<video id="video" width="320" height="240" autoplay></video>
<button id="snap">Snap Photo</button>
<canvas id="canvas" width="320" height="240"></canvas>

<script>
// Grab elements, create settings, etc.
var video = document.getElementById('video');

// Get access to the camera!
if(navigator.mediaDevices && navigator.mediaDevices.getUserMedia) {
    // Not adding `{ audio: true }` since we only want video now
    navigator.mediaDevices.getUserMedia({ video: true }).then(function(stream) {
        //video.src = window.URL.createObjectURL(stream);
        //video.play();
        video.srcObject=stream;
        video.play();
    });
}

// Elements for taking the snapshot
var canvas = document.getElementById('canvas');
var context = canvas.getContext('2d');
var video = document.getElementById('video');
var image = canvas.toDataURL("image/png");
IPython.notebook.kernel.execute("image = '" + image + "'")

// Trigger photo take
document.getElementById("snap").addEventListener("click", function() {
	context.drawImage(video, 0, 0, 320, 240);
    var myCanvas = document.getElementById('canvas');
    var image = myCanvas.toDataURL("image/png");
    IPython.notebook.kernel.execute("print('testing')")
    IPython.notebook.kernel.execute("imageWebCam = '" + image + "'")
});
</script>

"""

def show_bboxes_webcam(model):
    img = np.array(Image.open(io.BytesIO(base64.b64decode(imageWebCam.split(',')[1]))))[:,:,0:3]
    img = tf.image.resize(img, (256,256))
    show_img(img, model)
    
HTML(main_text)